# Examining SQL for Optimization

In [1]:
links = NewsLink.objects.all()

In [2]:
print(links.query)

SELECT "organizer_newslink"."id", "organizer_newslink"."title", "organizer_newslink"."slug", "organizer_newslink"."pub_date", "organizer_newslink"."link", "organizer_newslink"."startup_id" FROM "organizer_newslink" ORDER BY "organizer_newslink"."pub_date" DESC


In [3]:
print(
    NewsLink.objects
    .select_related('startup')
    .query
)

SELECT "organizer_newslink"."id", "organizer_newslink"."title", "organizer_newslink"."slug", "organizer_newslink"."pub_date", "organizer_newslink"."link", "organizer_newslink"."startup_id", "organizer_startup"."id", "organizer_startup"."name", "organizer_startup"."slug", "organizer_startup"."description", "organizer_startup"."founded_date", "organizer_startup"."contact", "organizer_startup"."website" FROM "organizer_newslink" INNER JOIN "organizer_startup" ON ("organizer_newslink"."startup_id" = "organizer_startup"."id") ORDER BY "organizer_newslink"."pub_date" DESC


## Prefetch SQL

In [4]:
posts = Post.objects.all()
posts_startups = (
    Post.objects.prefetch_related(
        'startups',
    )
)

In [5]:
list(posts_startups) == list(posts)

True

In [6]:
str(posts.query) == str(posts_startups.query)

True

In [7]:
print(posts_startups.query)

SELECT "blog_post"."id", "blog_post"."title", "blog_post"."slug", "blog_post"."text", "blog_post"."pub_date" FROM "blog_post" ORDER BY "blog_post"."pub_date" DESC, "blog_post"."title" ASC


In [8]:
from django.db import connection
from pprint import pprint

In [9]:
print(connection.queries[-1]['sql'])

SELECT "blog_post"."id", "blog_post"."title", "blog_post"."slug", "blog_post"."text", "blog_post"."pub_date" FROM "blog_post" ORDER BY "blog_post"."pub_date" DESC, "blog_post"."title" ASC


In [10]:
print(connection.queries[-2]['sql'])
# a prefetch causes a new query

SELECT ("blog_post_startups"."post_id") AS "_prefetch_related_val_post_id", "organizer_startup"."id", "organizer_startup"."name", "organizer_startup"."slug", "organizer_startup"."description", "organizer_startup"."founded_date", "organizer_startup"."contact", "organizer_startup"."website" FROM "organizer_startup" INNER JOIN "blog_post_startups" ON ("organizer_startup"."id" = "blog_post_startups"."startup_id") WHERE "blog_post_startups"."post_id" IN (1, 2) ORDER BY "organizer_startup"."name" ASC


In [11]:
from django.db import reset_queries
reset_queries()

In [12]:
pprint(connection.queries)

[]


In [13]:
posts = Post.objects.prefetch_related('startups')

In [14]:
# the queryset never evaluated!
pprint(connection.queries)

[]


In [15]:
posts = list(posts)  # force evaluation

In [16]:
# two queries:
#     the first for Post,
#     the second for Startups related to the Post
pprint(connection.queries)

[{'sql': 'SELECT "blog_post"."id", "blog_post"."title", "blog_post"."slug", '
         '"blog_post"."text", "blog_post"."pub_date" FROM "blog_post" ORDER BY '
         '"blog_post"."pub_date" DESC, "blog_post"."title" ASC',
  'time': '0.001'},
 {'sql': 'SELECT ("blog_post_startups"."post_id") AS '
         '"_prefetch_related_val_post_id", "organizer_startup"."id", '
         '"organizer_startup"."name", "organizer_startup"."slug", '
         '"organizer_startup"."description", '
         '"organizer_startup"."founded_date", "organizer_startup"."contact", '
         '"organizer_startup"."website" FROM "organizer_startup" INNER JOIN '
         '"blog_post_startups" ON ("organizer_startup"."id" = '
         '"blog_post_startups"."startup_id") WHERE '
         '"blog_post_startups"."post_id" IN (1, 2) ORDER BY '
         '"organizer_startup"."name" ASC',
  'time': '0.001'}]


In [17]:
reset_queries()
# three queries:
#     first for Post,
#     the second for Startups associated with those Posts
#     and then for the Tags associated with the Startups
posts = list(
    Post.objects.prefetch_related(
        'startups__tags'
    ),
)
posts_conn = connection.queries
pprint(posts_conn)

[{'sql': 'SELECT "blog_post"."id", "blog_post"."title", "blog_post"."slug", '
         '"blog_post"."text", "blog_post"."pub_date" FROM "blog_post" ORDER BY '
         '"blog_post"."pub_date" DESC, "blog_post"."title" ASC',
  'time': '0.001'},
 {'sql': 'SELECT ("blog_post_startups"."post_id") AS '
         '"_prefetch_related_val_post_id", "organizer_startup"."id", '
         '"organizer_startup"."name", "organizer_startup"."slug", '
         '"organizer_startup"."description", '
         '"organizer_startup"."founded_date", "organizer_startup"."contact", '
         '"organizer_startup"."website" FROM "organizer_startup" INNER JOIN '
         '"blog_post_startups" ON ("organizer_startup"."id" = '
         '"blog_post_startups"."startup_id") WHERE '
         '"blog_post_startups"."post_id" IN (1, 2) ORDER BY '
         '"organizer_startup"."name" ASC',
  'time': '0.001'},
 {'sql': 'SELECT ("organizer_startup_tags"."startup_id") AS '
         '"_prefetch_related_val_startup_id", "organiz

In [18]:
reset_queries()
# the following query is redundant
posts = list(
    Post.objects.prefetch_related(
        'startups',
        'startups__tags'
    ),
)
pprint(connection.queries)

[{'sql': 'SELECT "blog_post"."id", "blog_post"."title", "blog_post"."slug", '
         '"blog_post"."text", "blog_post"."pub_date" FROM "blog_post" ORDER BY '
         '"blog_post"."pub_date" DESC, "blog_post"."title" ASC',
  'time': '0.001'},
 {'sql': 'SELECT ("blog_post_startups"."post_id") AS '
         '"_prefetch_related_val_post_id", "organizer_startup"."id", '
         '"organizer_startup"."name", "organizer_startup"."slug", '
         '"organizer_startup"."description", '
         '"organizer_startup"."founded_date", "organizer_startup"."contact", '
         '"organizer_startup"."website" FROM "organizer_startup" INNER JOIN '
         '"blog_post_startups" ON ("organizer_startup"."id" = '
         '"blog_post_startups"."startup_id") WHERE '
         '"blog_post_startups"."post_id" IN (1, 2) ORDER BY '
         '"organizer_startup"."name" ASC',
  'time': '0.001'},
 {'sql': 'SELECT ("organizer_startup_tags"."startup_id") AS '
         '"_prefetch_related_val_startup_id", "organiz

In [19]:
pprint([info["sql"] for info in connection.queries])

['SELECT "blog_post"."id", "blog_post"."title", "blog_post"."slug", '
 '"blog_post"."text", "blog_post"."pub_date" FROM "blog_post" ORDER BY '
 '"blog_post"."pub_date" DESC, "blog_post"."title" ASC',
 'SELECT ("blog_post_startups"."post_id") AS "_prefetch_related_val_post_id", '
 '"organizer_startup"."id", "organizer_startup"."name", '
 '"organizer_startup"."slug", "organizer_startup"."description", '
 '"organizer_startup"."founded_date", "organizer_startup"."contact", '
 '"organizer_startup"."website" FROM "organizer_startup" INNER JOIN '
 '"blog_post_startups" ON ("organizer_startup"."id" = '
 '"blog_post_startups"."startup_id") WHERE "blog_post_startups"."post_id" IN '
 '(1, 2) ORDER BY "organizer_startup"."name" ASC',
 'SELECT ("organizer_startup_tags"."startup_id") AS '
 '"_prefetch_related_val_startup_id", "organizer_tag"."id", '
 '"organizer_tag"."name", "organizer_tag"."slug" FROM "organizer_tag" INNER '
 'JOIN "organizer_startup_tags" ON ("organizer_tag"."id" = '
 '"organizer_

In [20]:
def sql_is_equal(q1, q2):
    return [info["sql"] for info in q1] == [info["sql"] for info in q2]

In [21]:
sql_is_equal(posts_conn, connection.queries)

True

## Prefetch Objects

In [22]:
reset_queries()
posts = list(
    Post.objects.prefetch_related(
        Prefetch(
            'startups__tags',
        ),
    )
)
sql_is_equal(posts_conn, connection.queries)

True

In [23]:
reset_queries()
posts = list(
    Post.objects.prefetch_related(
        Prefetch(
            'startups__tags',
            queryset=Tag.objects.all(),
            to_attr='cached_tags',
        ),
    )
)
sql_is_equal(posts_conn, connection.queries)

True

In [24]:
# make sure none of the code calls the database!
reset_queries()

In [ ]:
posts

In [25]:
django_training = posts[0]

In [26]:
django_training.startups.all()

<QuerySet [<Startup: JamBon Software>]>

In [27]:
type(django_training.startups.all())

django.db.models.query.QuerySet

In [28]:
jambon_software = django_training.startups.all()[0]
jambon_software.cached_tags

[<Tag: Consulting>, <Tag: Django>, <Tag: Education>]

In [29]:
type(jambon_software.cached_tags)

list

In [30]:
# no database calls!
pprint(connection.queries)

[]


In [31]:
jambon_software.tags.all()

<QuerySet [<Tag: Consulting>, <Tag: Django>, <Tag: Education>]>

In [32]:
pprint(connection.queries)

[{'sql': 'SELECT "organizer_tag"."id", "organizer_tag"."name", '
         '"organizer_tag"."slug" FROM "organizer_tag" INNER JOIN '
         '"organizer_startup_tags" ON ("organizer_tag"."id" = '
         '"organizer_startup_tags"."tag_id") WHERE '
         '"organizer_startup_tags"."startup_id" = 1 ORDER BY '
         '"organizer_tag"."name" ASC  LIMIT 21',
  'time': '0.001'}]


In [33]:
reset_queries()
startups = list(
    Startup.objects.prefetch_related(
        Prefetch(
            'blog_posts',
            queryset=(
                # could use select_related here!
                Post.objects.all()
            ),
            to_attr='cached_posts',
        ),
        Prefetch(
            'cached_posts__tags',
            to_attr='cached_post_tags',
        ),
        Prefetch(
            'tags',
            to_attr='cached_tags',
        ),
        Prefetch(
            'cached_tags__startup_set',
        ),
    )
)
len(connection.queries)

5

In [34]:
reset_queries()
startups

[<Startup: JamBon Software>, <Startup: True Tickets>]

In [35]:
jambon_software = startups[0]

In [36]:
jambon_software.cached_posts

[<Post: Django Training on 2019-08-22>, <Post: Example Post on 2019-08-22>]

In [37]:
django_training = jambon_software.cached_posts[0]

In [38]:
django_training.cached_post_tags

[<Tag: Django>, <Tag: Education>]

In [39]:
jambon_software.cached_tags

[<Tag: Consulting>, <Tag: Django>, <Tag: Education>]

In [40]:
django = jambon_software.cached_tags[1]
django.startup_set.all()

<QuerySet [<Startup: JamBon Software>, <Startup: True Tickets>]>

In [41]:
for startup in startups:
    print(
        '{} has the following competitors:'
        .format(startup))
    for tag in startup.cached_tags:
        for competitor in tag.startup_set.all():
            if competitor.pk != startup.pk:
                print(
                    '    {}'
                    .format(competitor))

JamBon Software has the following competitors:
    True Tickets
True Tickets has the following competitors:
    JamBon Software


In [42]:
pprint(connection.queries)

[]


## Limiting Fields

In [43]:
list(Tag.objects.values('name', 'slug'))

[{'name': 'Business-to-Business', 'slug': 'business-to-business'},
 {'name': 'Consulting', 'slug': 'consulting'},
 {'name': 'Distributed Systems', 'slug': 'distributed-systems'},
 {'name': 'Django', 'slug': 'django'},
 {'name': 'Education', 'slug': 'education'},
 {'name': 'IBM Blockchain', 'slug': 'ibm-blockchain'},
 {'name': 'Mobile', 'slug': 'mobile'},
 {'name': 'Video Games', 'slug': 'video-games'},
 {'name': 'Web', 'slug': 'web'}]

In [44]:
print(Tag.objects.values('name', 'slug').query)

SELECT "organizer_tag"."name", "organizer_tag"."slug" FROM "organizer_tag" ORDER BY "organizer_tag"."name" ASC


In [45]:
list(Tag.objects.values_list('name', flat=True))

['Business-to-Business',
 'Consulting',
 'Distributed Systems',
 'Django',
 'Education',
 'IBM Blockchain',
 'Mobile',
 'Video Games',
 'Web']

In [46]:
print(Tag.objects.values_list('name', flat=True).query)

SELECT "organizer_tag"."name" FROM "organizer_tag" ORDER BY "organizer_tag"."name" ASC


In [47]:
print(Tag.objects.all().query)

SELECT "organizer_tag"."id", "organizer_tag"."name", "organizer_tag"."slug" FROM "organizer_tag" ORDER BY "organizer_tag"."name" ASC


In [48]:
print(Tag.objects.defer('slug').query)

SELECT "organizer_tag"."id", "organizer_tag"."name" FROM "organizer_tag" ORDER BY "organizer_tag"."name" ASC


In [49]:
print(Tag.objects.defer('id').query)

SELECT "organizer_tag"."id", "organizer_tag"."name", "organizer_tag"."slug" FROM "organizer_tag" ORDER BY "organizer_tag"."name" ASC


In [50]:
from django.core.exceptions import FieldDoesNotExist
try:
    tags = list(Tag.objects.defer('pk'))
except FieldDoesNotExist as e:
    print(e)

Tag has no field named 'pk'


In [52]:
list(Tag.objects.only('name'))

[<Tag: Business-to-Business>,
 <Tag: Consulting>,
 <Tag: Distributed Systems>,
 <Tag: Django>,
 <Tag: Education>,
 <Tag: IBM Blockchain>,
 <Tag: Mobile>,
 <Tag: Video Games>,
 <Tag: Web>]

In [53]:
str(Tag.objects.defer('slug').query) == str(Tag.objects.only('name').query)

True

In [54]:
print(Tag.objects.only('name').values().query)

SELECT "organizer_tag"."id", "organizer_tag"."name", "organizer_tag"."slug" FROM "organizer_tag" ORDER BY "organizer_tag"."name" ASC


In [55]:
try:
    tags = Tag.objects.values().only('name')
except TypeError as e:
    print(e)

Cannot call only() after .values() or .values_list()
